# PowerShell dicas e truques

Conteudo apresentado no **DevWeek** 01/12/2020 [live](https://www.youtube.com/watch?v=JF0p7rlHYU0) do **Canal dotNET**.  

## Requisito ⚠

[PowerShell 7](https://github.com/PowerShell/PowerShell/releases/tag/v7.1.0/)

~~[WSL](https://docs.microsoft.com/pt-br/windows/wsl/install-win10)~~

[Windows Terminal](https://www.microsoft.com/pt-br/p/windows-terminal/9n0dx20hk701?activetab=pivot:overviewtab)

[Visual Studio Code](https://code.visualstudio.com/)

[.Net Core 3.1](https://dotnet.microsoft.com/download/dotnet-core)

[.Net interactive](https://github.com/dotnet/interactive)

______

- Consumo de API.

1. Exemplo com querystring
2. Exemplo passando os parametros no body

In [ ]:
Add-Type -AssemblyName System.Web
$urlParam = @{
    urlPage  = "https://cat-fact.herokuapp.com"
    uriPage = "/facts/random"
    method   = "Get"
}
$bodyParam = [Ordered]@{
    animal_type = @("horse", "cat", "dog", "snail") | Get-Random 
    amount      = 1
}

$qCollection = [System.Web.HttpUtility]::ParseQueryString([String]::Empty)
foreach ($key in $bodyParam.Keys) {
    $qCollection.Add($key, $($bodyParam."$key") )
}
$uriBuilder = [System.UriBuilder] -join ($urlParam.urlPage, $urlParam.uriPage)
$uriBuilder.Query = $qCollection.ToString()
$returnRequest = Invoke-RestMethod -Uri "$($uriBuilder.Uri)" -Method $urlParam.method
$returnRequest

In [ ]:
$restParam = @{
    Uri  = "https://jsonplaceholder.typicode.com/posts"
    Method   = "Post"
    Headers = [Ordered]@{"Content-Type"="application/json";"charset"="UTF-8"}
    Body = @{"title"="Sample";"body"="Ola Mundo";"userId"=1} | ConvertTo-Json
}
$result = Invoke-RestMethod @restParam
$result

- Utilizando Alias

Alias facilitam muito a vida de qualquer desenvolvedor, no __PowerShell__ temos a possibilidade de criar os nossos próprios __Alias__ para os comandos que utilizamos.


In [ ]:
New-Alias -Name dn -Value "dotnet.exe"
dn new
function dotnetnew { 
    dotnet.exe new $args
}
New-Alias -Name dnn -Value dotnetnew 
dnn console -n aplicacaoconsole


Dica: O alias da maneira que criamos **existe** enquanto a sua sessão PowerShell estiver **ativa**, para persistir o valor dos alias definidos você deve exportar os alias ou incluir a criação dos alias no seu arquivo de **profile**.

In [14]:
$PROFILE | Select-Object * | Format-List

_________________

- PowerShell com a commandline do VSCode

Utilizando o PowerShell para instalar extensões no Visual Studio Code.

In [ ]:
$contentFile = @"
AwesomeAutomationTeam.azureautomation 
Azure.vscode-bicep
bierner.markdown-preview-github-styles
cssho.vscode-svgviewer
DavidAnson.vscode-markdownlint
docsmsft.docs-markdown
DotJoshJohnson.xml
dunstontc.vscode-docker-syntax
henoc.svgeditor
jock.svg
ms-azuretools.vscode-azurefunctions
ms-azuretools.vscode-azurevirtualmachines
ms-azuretools.vscode-cosmosdb
ms-azuretools.vscode-docker
ms-azuretools.vscode-logicapps
ms-dotnettools.csharp
ms-dotnettools.vscode-dotnet-runtime
ms-python.python
ms-toolsai.jupyter
ms-vscode-remote.remote-containers
ms-vscode-remote.remote-ssh
ms-vscode-remote.remote-ssh-edit
ms-vscode-remote.remote-wsl
ms-vscode-remote.vscode-remote-extensionpack
ms-vscode.azure-account
ms-vscode.azurecli
ms-vscode.powershell
ms-vscode.powershell-preview
ms-vsliveshare.vsliveshare
ms-vsonline.vsonline
msazurermtools.azurerm-vscode-tools
redhat.vscode-yaml
shd101wyy.markdown-preview-enhanced
SimonSiefke.svg-preview
Tyriar.shell-launcher
"@
New-item "$home\desktop" -Name extensionsvscode.txt -itemtype file -value $contentFile

Get-Content "$home\desktop\extensionsvscode.txt" | ForEach-Object {
   code-insiders.cmd  --install-extension  $_
}

- Utilizando codigo C# no PowerShell

Dando novos poderes para a sua linha de comando. 

1. Exemplo com codigo C# e adicionando a sessão
2. Exemplo Definindo uma função PowerShell e utilizando namespaces

In [ ]:
#!pwsh
$codigoCsharp = @"
using System;
using System.Net;
using System.Net.NetworkInformation;
using System.Text;

namespace Examples.System.Net.NetworkInformation.PingTest
{
    public class PingExample
    {
        // args[0] can be an IPaddress or host name.
        public static void Main (string[] args)
        {
            Ping pingSender = new Ping ();
            PingOptions options = new PingOptions ();

            // Use the default Ttl value which is 128,
            // but change the fragmentation behavior.
            options.DontFragment = true;

            // Create a buffer of 32 bytes of data to be transmitted.
            string data = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa";
            byte[] buffer = Encoding.ASCII.GetBytes (data);
            int timeout = 120;
            PingReply reply = pingSender.Send (args[0], timeout, buffer, options);
            if (reply.Status == IPStatus.Success)
            {
                Console.WriteLine ("Address: {0}", reply.Address.ToString ());
                
                Console.WriteLine ("RoundTrip time: {0}", reply.RoundtripTime);
                Console.WriteLine ("Time to live: {0}", reply.Options.Ttl);
                Console.WriteLine ("Don't fragment: {0}", reply.Options.DontFragment);
                Console.WriteLine ("Buffer size: {0}", reply.Buffer.Length);
            }
        }
    }
}
"@
Add-Type -TypeDefinition $codigoCsharp -Language CSharp
Invoke-Expression "[Examples.System.Net.NetworkInformation.PingTest.PingExample]::Main(`"1.1.1.1`")"
#[HelloWorld.Program]::Main()

In [ ]:
#!pwsh
function Test-Ping {
    [CmdletBinding()]
    param (
        [Parameter(Mandatory, Position = 0)]
        [string]
        $IPaddress
    )
    $pingSender = New-Object System.Net.NetworkInformation.Ping
    $pingOptions = [System.Net.NetworkInformation.PingOptions]::new()
    $pingOptions.DontFragment = $true
    $data = -join (1..32 | % {"a"})
    $bufferData = @([System.Text.ASCIIEncoding]::ASCII.GetBytes($data))
    [int]$timeout = 120
    [System.Net.NetworkInformation.PingReply]$pingReplay = $pingSender.Send($IPaddress,$timeout,$bufferData,$pingOptions)
    if ($pingReplay.Status -eq "Success") {
        [PSCustomObject]@{
            "Address" = $pingReplay.Address
            "RoundTrip time" = $pingReplay.RoundtripTime
            "TTL" =  $pingReplay.Options.Ttl
            "Fragment" = $pingReplay.Options.DontFragment
            "BufferSize" = $pingReplay.Buffer.Length
        } | Format-List
    }
}

- Windows Terminal

Iniciando o seu Windows Terminal utilizando uma função no __PowerShell__.

In [3]:
function New-WinTerminal {
    wt.exe  new-tab  `; split-pane -p "Windows PowerShell" `; split-pane -H wsl.exe
}
New-WinTerminal

## Referência ✅

[SDK PowerShell](https://docs.microsoft.com/en-us/dotnet/api/system.management.automation?view=powershellsdk-7.0.0)

[PowerShell Docs](https://docs.microsoft.com/pt-br/powershell/scripting/how-to-use-docs?view=powershell-7.1)

[PowerShell Team GitHub](https://github.com/PowerShell)

[Pester](https://pester-docs.netlify.app/docs/quick-start#creating-a-pester-test)

[PowerShell 101 free book](https://leanpub.com/powershell101)

[WindowsTerminal Docs](https://docs.microsoft.com/pt-br/windows/terminal/)

[WindowsTerminal GitHub](https://github.com/microsoft/terminal)

[WSL Docs](https://docs.microsoft.com/en-us/windows/wsl/about)

[WSL Release Notes](https://docs.microsoft.com/en-us/windows/wsl/release-notes)

